In [ ]:
# Business Scenario
# A smart city power department wants to predict tomorrow’s electricity demand (in MW) using the last 7 days of hourly consumption data.
# This helps to:
# Avoid power shortages ⚡
# Optimize power generation scheduling
# Reduce operational costs
# 🔹 Sample Dataset (Simplified – Daily Average)
 
 

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [5]:
df = pd.read_csv('electricity_demand.csv')
df.head()

,Day,Electricity Demand (MW)
0,1,320.0
1,2,340.0
2,3,360.0
3,4,355.0
4,5,370.0


In [6]:
X = df[['Day']].values
y = df[['Electricity Demand (MW)']].values

window_size = 7

mean = X.mean()
std = X.std()
X_norm = (X - mean) / std
y_norm = (y - mean) / std

# Reshape for RNN: (batch, time_steps, features)
X_norm = X_norm[..., np.newaxis]   # (samples, 7, 1)
y_norm = y_norm[..., np.newaxis]   # (samples, 1)

# Train/validation split
split = int(0.8 * len(X_norm))
X_train, X_val = X_norm[:split], X_norm[split:]
y_train, y_val = y_norm[:split], y_norm[split:]

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def build_lstm_model():
    model = Sequential([
        LSTM(
            units = 32,
            activation = 'tanh',
            recurrent_activation = 'sigmoid',
            input_shape = (window_size, 1)
        ),
        Dense(1)
    ])
    return model

lstm_model = build_lstm_model()
lstm_model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate = 1.0)

lstm_model.compile(
    optimizer = optimizer,
    loss = 'mse',
    metrics = ['mae']
)

history_lstm = lstm_model.fit(
    X_train, y_train,
    epochs = 500,
    batch_size = 10,
    validation_data = (X_val, y_val),
    verbose = 0
)

print("LSTM Final train loss:", history_lstm.history['loss'][-1])
print("LSTM Final val loss:", history_lstm.history['val_loss'][-1])

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                  │ (None, 32)             │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,385 (17.13 KB)

 Trainable params: 4,385 (17.13 KB)

 Non-trainable params: 0 (0.00 B)

LSTM Final train loss: 1.370906801412275e-07
LSTM Final val loss: 275.8851623535156


In [27]:
from tensorflow.keras.layers import GRU

def build_gru_model():
    model = Sequential([
        GRU(
            units = 32,
            activation = 'tanh',
            recurrent_activation = 'sigmoid',
            input_shape = (window_size, 1)
        ),
        Dense(1)
    ])
    return model

gru_model = build_gru_model()
gru_model.summary()

gru_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.1),
    loss = 'mse',
    metrics = ['mae']
)

history_gru = gru_model.fit(
    X_train, y_train,
    epochs = 500,
    batch_size = 10,
    validation_data = (X_val, y_val),
    verbose = 0
)

print("GRU final train loss", history_gru.history['loss'][-1])
print("GRU final val loss:", history_gru.history['val_loss'][-1])

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_6 (GRU)                     │ (None, 32)             │         3,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,393 (13.25 KB)

 Trainable params: 3,393 (13.25 KB)

 Non-trainable params: 0 (0.00 B)

GRU final train loss 77.22125244140625
GRU final val loss: 702.5137329101562


In [28]:
last_week = y[-window_size:]
last_week_norm = ((last_week - mean) / std).reshape(1, window_size, 1)

pred_lstm_norm = lstm_model.predict(last_week_norm, verbose=0)[0, 0]
pred_gru_norm = gru_model.predict(last_week_norm, verbose=0)[0, 0]

pred_lstm = pred_lstm_norm * std + mean
pred_gru = pred_gru_norm * std + mean

print("Last 7 days traffic:", np.round(last_week, 1))
print(f"LSTM predicted next day traffic: {pred_lstm:.1f} visits")
print(f"GRU  predicted next day traffic: {pred_gru:.1f} visits")

Last 7 days traffic: [[320.]
 [340.]
 [360.]
 [355.]
 [370.]
 [390.]
 [410.]]
LSTM predicted next day traffic: 54.9 visits
GRU  predicted next day traffic: 13.0 visits
